# Modeling dynamics of FS Peptide
This example shows a typical, basic usage of MSMBuilder to model dynamics of a protein system. 

### MSMBuilder includes example datasets
The following cell downloads an example protein dataset to ~/msmbuilder_data. In practice, you will generate your own dataset by running molecular dynamics (MD) on your system of interest. MSMBuilder does not run MD.

In [ ]:
# Download example dataset
from msmbuilder.example_datasets import FsPeptide
fs_peptide = FsPeptide()
fs_peptide.cache()
import os
os.chdir(fs_peptide.data_dir)

### The `dataset` object
MD datasets are usually quite large. It doesn't make sense to load everything into memory at once. The `dataset` object lazily-loads trajectories as they are needed. Below, we create a dataset out of the 28 `*.xtc` files we downloaded above.

In [ ]:
from msmbuilder.dataset import dataset
xyz = dataset("*.xtc", topology='fs_peptide.pdb')
len(xyz)

### Featurization
The raw `(x, y, z)` coordinates from the simulation do not respect the translational and rotational symmetry of our problem. A `Featurizer` transforms cartesian coordinates into other representations. Here we use the `DihedralFeaturizer` to turn our data into phi and psi dihedral angles. Observe that the `264*3`-dimensional space is reduced to 84 dimensions

In [ ]:
from msmbuilder.featurizer import DihedralFeaturizer
featurizer = DihedralFeaturizer(types=['phi', 'psi'])
diheds = xyz.fit_transform_with(featurizer, 'diheds/', fmt='dir-npy')

print(xyz[0].xyz.shape)
print(diheds[0].shape)

### Intermediate kinetic model: tICA
`tICA` is similar to principal component analysis (see "tICA vs. PCA" example). Note that the 84-dimensional space is reduced to 4 dimensions.

In [ ]:
from msmbuilder.decomposition import tICA
tica_model = tICA(lag_time=1, n_components=4)
# fit and transform can be done in seperate steps:
tica_model = diheds.fit_with(tica_model)
tica_trajs = diheds.transform_with(tica_model, 'ticas/', fmt='dir-npy')

print(diheds[0].shape)
print(tica_trajs[0].shape)

### tICA Heatmap
We can histogram our data projecting along the two slowest degrees of freedom (as found by tICA)

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
txx = np.concatenate(tica_trajs)
plt.hexbin(txx[:,0], txx[:,1], bins='log', mincnt=1)

### Clustering
Conformations need to be clustered into states (sometimes written as microstates). We cluster based on the tICA projections to group conformations that interconvert rapidly. Note that we transform our trajectories from the 4-dimensional tICA space into a 1-dimensional cluster index.

In [ ]:
from msmbuilder.cluster import MiniBatchKMeans
clusterer = MiniBatchKMeans(n_clusters=100)
clustered_trajs = tica_trajs.fit_transform_with(clusterer, 'kmeans/', fmt='dir-npy')

print(tica_trajs[0].shape)
print(clustered_trajs[0].shape)

In [ ]:
plt.hexbin(txx[:,0], txx[:,1], bins='log', mincnt=1)
plt.scatter(clusterer.cluster_centers_[:,0],
            clusterer.cluster_centers_[:,1], 
            s=100, c='w')

### MSM
We can construct an MSM from the labeled trajectories

In [ ]:
from msmbuilder.msm import MarkovStateModel
from msmbuilder.utils import dump
msm = MarkovStateModel(lag_time=5)
msm.fit(clustered_trajs)

In [ ]:
plt.hexbin(txx[:,0], txx[:,1], bins='log', mincnt=1, cmap="Greys")
plt.scatter(clusterer.cluster_centers_[:,0],
            clusterer.cluster_centers_[:,1],
            s=1e4 * msm.populations_, # size by population
            c=msm.left_eigenvectors_[:,1], # color by eigenvector
            cmap="RdBu") 
plt.colorbar(label='First dynamical eigenvector')
plt.xlabel('tIC 1')
plt.ylabel('tIC 2')
plt.tight_layout()

## Macrostate Model

In [ ]:
from msmbuilder.lumping import PCCAPlus
pcca = PCCAPlus.from_msm(msm, n_macrostates=5)
macro_trajs = pcca.transform(clustered_trajs)

In [ ]:
plt.hexbin(txx[:,0], txx[:,1], bins='log', mincnt=1, cmap="Greys")
plt.scatter(clusterer.cluster_centers_[:,0],
            clusterer.cluster_centers_[:,1],
            s=100,
            c=pcca.microstate_mapping_,
      )
plt.xlabel('tIC 1')
plt.ylabel('tIC 2')